In [5]:
!apt-get update &>/dev/null
!apt-get install python3.8  &>/dev/null
!update-alternatives --install /usr/bin/python python /usr/bin/python3.8 0 
!update-alternatives --list python

/usr/bin/python3.8


In [2]:
from collections import OrderedDict
import numpy as np

GROUPS=['A','B']
ZEROS=np.zeros_like(GROUPS, dtype=np.float64)
ONES=np.ones_like(GROUPS, dtype=np.float64)

In [42]:
!pip install japanize-matplotlib nptyping &>/dev/null && wait $PID # pyplot用日本語フォント，Numpy用型ヒント
from matplotlib import pylab as plt
import japanize_matplotlib

import numpy as np
import pandas as pd

from pprint import pprint as pprint
from typing import Callable, List, Dict, Tuple, Any
from nptyping import NDArray, Float

Matrix=NDArray[(len(GROUPS),),Float[64]]

class SIRParameter:
    def __init__(self, name, init_value: Matrix, single=True):
        self.name=name
        self.matrixes=[np.copy(init_value)]
        self.t=0

    def __getitem__(self, t):
        if(self.single):
            return self.matrixes[0]
        else:
            return self.matrixes[self.t]

class SIRVariable:
    def __init__(self, name, init_value: Matrix, outputs):
        self.name=name
        self.matrixes=[np.copy(init_value)]
        self.t=0
        self.outputs=outputs

    def shape(self):
        return self.matrixes[self.t].shape

    def eq_shape(self, x):
        if(not isinstance(x, np.ndarray)):
            return True

        if(self.shape()==x.shape):
            return True
        else:
            raise ValueError("The shapes is NOT eqqual.: "+self.name+".shape()!="+x.name+".shape(). "
            +self.name+".shape()"+self.shape()+", "
            +x.name+".shape()"+x.shape)

    def __getitem__(self, t):
        if(t < len(self.matrixes)):
            return self.matrixes[t]
        else:
            print("WARN: t:",t," != len(self.matrixes):",len(self.matrixes))
            self.matrixes.insert(t+1, x)
            return self.matrixes[t]

    def __setitem__(self, t, x: Matrix):
        self.eq_shape(x)
        if(t < len(self.matrixes)):
            self.matrixes[t]=x
        else:
            self.matrixes.insert(t+1, x)

    def __sub__(self, x: Matrix):
        self.eq_shape(x)
        return self[self.t]-x

    def __add__(self, x: Matrix):
        self.eq_shape(x)
        return self[self.t]+x

    def __mul__(self, x: Matrix):
        self.eq_shape(x)
        print("x",x)
        return np.multiply(self[self.t], x)

    def __rmatmul__(self, x: Matrix):
        self.eq_shape(x)
        return np.dot(self[self.t], x)

    def __iadd__(self, x: Matrix):
        self.eq_shape(x)
        self[self.t]+=x

    def __isub__(self, x: Matrix):
        self.eq_shape(x)
        self[self.t]-=x

class Model:
    def __init__(self):
        self.vars=OrderedDict()
    
    def add_var(self, var: SIRVariable):
        self.vars[var.name]=var

    def __getitem__(self, name: str):
        return self.vars[name]

    def calc(self, t):
        self.t=t
        if(0<t):
            self['S'][t]=self['S'][t-1]
            self['I'][t]=self['I'][t-1]
            self['R'][t]=self['R'][t-1]
        for var_name, var in self.vars.items():#元のSIR
            for target_var_name, outf in var.outputs.items():#先のSIRとf()
                var[t]-=outf(t, self, var[t])
                self.vars[target_var_name][t]+=outf(t, self, var[t])
                print(target_var_name,outf(t, self, var[t]))

class OutputFuncs:
    def const_prod_output(const):
        def const_prod_output_func(t, model, x: Matrix):
            print("t=",t)
            print("const=", const)
            print("x=",x)
            return x*const
        return const_prod_output_func

    def two_input_const_prod_output(x2_name, const):
        def two_input_const_prod_output_func(t, model, x: Matrix):
            print("t",t)
            print("const", const)
            print(x2_name+"["+str(t)+"]",model[x2_name][t])
            print("x=",x)
            return const*model[x2_name][t]*x
        return two_input_const_prod_output_func

In [43]:
def run(T_MAX=100):
    β=0.1
    γ=1/3
    model=Model()
    S=model.add_var(SIRVariable('S',init_value=np.array([1000.,0.]),outputs={'I':OutputFuncs.two_input_const_prod_output('I',β)}))
    I=model.add_var(SIRVariable('I',init_value=ONES,outputs={'R':OutputFuncs.const_prod_output(γ)}))
    R=model.add_var(SIRVariable('R',init_value=ZEROS,outputs={}))
    for t in range(T_MAX):
        model.calc(t)
run(100)

t 0
const 0.1
I[0] [1. 1.]
x= [1000.    0.]
t 0
const 0.1
I[0] [1. 1.]
x= [900.   0.]
t 0
const 0.1
I[0] [91.  1.]
x= [900.   0.]
I [8190.    0.]
t= 0
const= 0.3333333333333333
x= [91.  1.]
t= 0
const= 0.3333333333333333
x= [60.66666667  0.66666667]
t= 0
const= 0.3333333333333333
x= [60.66666667  0.66666667]
R [20.22222222  0.22222222]
t 1
const 0.1
I[1] [60.66666667  0.66666667]
x= [900.   0.]
t 1
const 0.1
I[1] [60.66666667  0.66666667]
x= [-4560.     0.]
t 1
const 0.1
I[1] [-2.76033333e+04  6.66666667e-01]
x= [-4560.     0.]
I [12587120.00000001        0.        ]
t= 1
const= 0.3333333333333333
x= [-2.76033333e+04  6.66666667e-01]
t= 1
const= 0.3333333333333333
x= [-1.84022222e+04  4.44444444e-01]
t= 1
const= 0.3333333333333333
x= [-1.84022222e+04  4.44444444e-01]
R [-6.13407407e+03  1.48148148e-01]
t 2
const 0.1
I[2] [-1.84022222e+04  4.44444444e-01]
x= [-4560.     0.]
t 2
const 0.1
I[2] [-1.84022222e+04  4.44444444e-01]
x= [-8395973.33333334        0.        ]
t 2
const 0.1
I[2] [

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:124: RuntimeWarning: overflow encountered in multiply
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:105: RuntimeWarning: invalid value encountered in subtract


 [       nan 0.00020049]
t= 20
const= 0.3333333333333333
x= [       nan 0.00020049]
R [           nan 6.68285911e-05]
t 21
const 0.1
I[21] [       nan 0.00020049]
x= [nan  0.]
t 21
const 0.1
I[21] [       nan 0.00020049]
x= [nan  0.]
t 21
const 0.1
I[21] [       nan 0.00020049]
x= [nan  0.]
I [nan  0.]
t= 21
const= 0.3333333333333333
x= [       nan 0.00020049]
t= 21
const= 0.3333333333333333
x= [       nan 0.00013366]
t= 21
const= 0.3333333333333333
x= [       nan 0.00013366]
R [          nan 4.4552394e-05]
t 22
const 0.1
I[22] [       nan 0.00013366]
x= [nan  0.]
t 22
const 0.1
I[22] [       nan 0.00013366]
x= [nan  0.]
t 22
const 0.1
I[22] [       nan 0.00013366]
x= [nan  0.]
I [nan  0.]
t= 22
const= 0.3333333333333333
x= [       nan 0.00013366]
t= 22
const= 0.3333333333333333
x= [           nan 8.91047881e-05]
t= 22
const= 0.3333333333333333
x= [           nan 8.91047881e-05]
R [          nan 2.9701596e-05]
t 23
const 0.1
I[23] [           nan 8.91047881e-05]
x= [nan  0.]
t 23
const